In [1]:
# from IPython.display import HTML

# HTML('''<script>
# code_show=true; 
# function code_toggle() {
#  if (code_show){
#  $('div.input').hide();
#  } else {
#  $('div.input').show();
#  }
#  code_show = !code_show
# } 
# $( document ).ready(code_toggle);
# </script>
# The raw code for this IPython notebook is by default hidden for easier reading.
# To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')




In [2]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.lines as mlines
import random
import json
from math import ceil
%matplotlib inline
import os
import matplotlib as mpl
from PIL import Image
from io import BytesIO
from bayescmd.abc import SummaryStats
from bayescmd.abc import import_actual_data
from bayescmd.abc import inputParse
import scipy.stats as stats
import statsmodels.api as sm


import os
from pathlib import Path
from distutils import dir_util
from pprint import pprint
import pickle

# BayesCMD packages 

from bayescmd.abc import import_actual_data
from bayescmd.abc import priors_creator

# Google BigQuery
from google.cloud import bigquery
%load_ext google.cloud.bigquery


from bayescmd.results_handling import get_output

mpl.rc('figure', dpi=300, figsize=(7.5,8))

mpl.rcParams["xtick.labelsize"]= 8

mpl.rcParams["ytick.labelsize"]= 8
    
mpl.rcParams["axes.labelsize"]= 10

mpl.rcParams["figure.titlesize"] = 12

STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD
STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD
STARTING AT: /home/buck06191/repos/Github/BayesCMD/bayescmd
 Looking for: BayesCMD


In [3]:
def TIFF_exporter(fig, fname, fig_dir='.', extra_artists=()):
    """
    Parameters
    ----------
    fig: matplotlib figure
    """

    # save figure
    # (1) save the image in memory in PNG format
    # png1 = BytesIO()
    fig.savefig(os.path.join(fig_dir, '{}.png'.format(fname)), format='png', bbox_inches='tight', bbox_extra_artists=extra_artists,
                dpi=300, transparent=False)

#     # (2) load this image into PIL
#     png2 = Image.open(png1)

#     # (3) save as TIFF
#     png2.save(os.path.join(fig_dir, '{}.tiff'.format(fname)),
#               compression='tiff_deflate')
#     png1.close()
    return True

In [4]:
# Explicitly use service account credentials by specifying the private
# key file. All clients in google-cloud-python have this helper.
client = bigquery.Client.from_service_account_json(
    "../../gcloud/hypothermia-auth.json"
)

In [5]:
def generate_posterior_query(project, dataset, model, distance, parameters, limit=50000):
    unpacked_params = ",\n".join(parameters)
    histogram_query = """
SELECT
    {unpacked_params},
    {distance},
    idx
FROM
  `{project}.{dataset}.{model}`
ORDER BY
  {distance} ASC
LIMIT
  {limit}
    """.format(project=project, dataset=dataset, model=model, unpacked_params=unpacked_params,distance=distance, limit=limit)
    return histogram_query

In [6]:
def load_configuration(model_version, dataset, verbose=False):
    current_file = Path(os.path.abspath(''))
    config_file = os.path.join(current_file.parents[2],
                              'config_files',
                               'abc',
                               'bp_hypothermia_{}'.format(model_version),
                               'bp_hypothermia_{}_config.json'.format(model_version)
                              )

    with open(config_file, 'r') as conf_f:
        conf = json.load(conf_f)

    params = conf['priors']

    input_path = os.path.join(current_file.parents[2],
                              'data',
                              'clean_hypothermia',
                              '{}_filtered_formatted.csv'.format(dataset.upper()))

    d0 = import_actual_data(input_path)

    targets = conf['targets']
    model_name = conf['model_name']
    inputs = conf['inputs']

    config = {
        "model_name": model_name,
        "targets": targets,
        "times": d0['t'],
        "inputs": inputs,
        "parameters": params,
        "input_path": input_path,
        "zero_flag": conf['zero_flag'],
    }
    
    if verbose:
        pprint(config)
        
    return config, d0

# Read in the posterior #


In [7]:
labels = {"t": "Time (sec)", 
 "HbO2": "$\Delta$HbO2 $(\mu M)$",
 "HHb": "$\Delta$HHb $(\mu M)$",
 "CCO": "$\Delta$CCO $(\mu M)$"}

signals=['HbO2', 'HHb','CCO']
ticker_step = [20, 10, 10, 10, 0.5]
colpal = sns.color_palette(n_colors=len(signals))

In [8]:
def get_runs(posterior, conf, n_repeats=50): 
    rand_selection = random.sample(range(posterior.shape[0]), n_repeats)
    outputs_list = []
    p_names = list(conf['parameters'].keys())
    posteriors = posterior[p_names].values
    d0 = import_actual_data(conf['input_path'])
    input_data = inputParse(d0, conf['inputs'])
    while len(outputs_list) < n_repeats:
        idx = rand_selection.pop()
        print("\tSample {}, idx:{}".format(len(outputs_list), idx))
        p = dict(zip(p_names, posteriors[idx]))
            
    
        _, output = get_output(
            conf['model_name'],
            p,
            conf['times'],
            input_data,
            d0,
            conf['targets'],
            distance="NRMSE",
            zero_flag=conf['zero_flag'])
        outputs_list.append(output)
    return outputs_list


## Generating posterior predictive ##

We can sample directly from the posterior to generate our posterior predictive.We then generate a variety of potentially useful summary statistics as well as the residuals, autocorrelation of the signals and autocorrelation of the residuals for each signal.

We also generate each summary statistic for the observed data so as to compare this with the posterior predictive distribution of these statistics.

In [9]:
configuration = {}
model_data_combos = {"LWP475": ["1","2","4"],
                    "LWP479": ["1_1", "2_1", "4_1"]}
for combo in [(m,d) for d, l in model_data_combos.items() for m in l]:
    print("Working on (bph{}, {})".format(*combo))
    model_number = combo[0]
    model_name = 'bph{}'.format(model_number)
    DATASET = combo[1]
    configuration[model_name] = {}
    
    configuration[model_name][DATASET] = {}
    config, d0 = load_configuration(model_number, DATASET)
    configuration[model_name][DATASET]['bayescmd_config'] = config
    configuration[model_name][DATASET]['original_data']= d0

    configuration[model_name][DATASET]['posterior_query'] = generate_posterior_query('hypothermia-bayescmd', 
                                                                                     DATASET, 
                                                                                     model_name, 
                                                                                     'NRMSE', 
                                                                                     list(configuration[model_name][DATASET]['bayescmd_config']['parameters'].keys()),
                                                                                     limit=5000)
    config = configuration[model_name][DATASET]['bayescmd_config']
    figPath = "/home/buck06191/Dropbox/phd/hypothermia/ABC/Figures/{}/{}/{}".format(model_name, DATASET, 'NRMSE')
    dir_util.mkpath(figPath)

    # Get posterior
    print("\tRunning SQL query")
    df_post = client.query(configuration[model_name][DATASET]['posterior_query']).to_dataframe()
    N=5000
    print("\tSampling from the posterior {} times.".format(N))
    
    outputs_list = get_runs(df_post, config, n_repeats=N)
    results = {}
    print("\n")
    
    for i, output in enumerate(outputs_list):
        results[i] = {}
        summary_creator = SummaryStats(output, config['targets'], config['zero_flag'], observed_data = d0)
        summary_creator.get_stats()
        results[i]['data'] = summary_creator.d0
        results[i]['residuals'] = summary_creator.residuals
        results[i]['stats'] = summary_creator.summary_stats

    resid_formatted = [{'Batch': i, 'Signal': j, 'Residuals': v, 'Time (sec)': idx+1} for i in results.keys(
    ) for j in results[i]['residuals'].keys() for idx, v in enumerate(results[i]['residuals'][j])]
    residuals = pd.DataFrame(resid_formatted)
    fig1, axes1 = plt.subplots(2,2, figsize=(7,7))
    fig2, axes2 = plt.subplots(2,2, figsize=(7,7))

    for ii, s in enumerate(config['targets']):
        signal_data=residuals[residuals['Signal']==s]['Residuals']
        ax1=axes1.flatten()[ii]
        sns.distplot(signal_data, ax=ax1)
        resid_mu, resid_sigma = np.mean(signal_data), np.std(signal_data)
        print("\t{}: Mean $(\mu$): {:.3g}\n\tStandard Deviation ($\sigma$): {:.3g}".format(s.upper(),resid_mu, resid_sigma))
        mean = ax1.axvline(resid_mu, color='k', label='Mean', linestyle='--')
        std = ax1.axvline(resid_mu-resid_sigma, color='g', label='Standard Deviation', linestyle='--')
        ax1.axvline(resid_mu+resid_sigma, color='g', linestyle='--')
        ax1.set_title("{}".format(s), fontsize=12)


        ax2=axes2.flatten()[ii]
        resid = signal_data.values
        sm.qqplot(resid, line='s',ax=ax2)
        ax2.axhline(0, color='k', linestyle='--')
        sample_mean = ax2.axhline(resid_mu, color='xkcd:orange', linestyle=':', label="Sample Mean")
        theoretical_mean = ax2.axvline(0, color='k', linestyle='--', label="Theoretical Mean")
        ax2.set_title("{}".format(s), fontsize=12)
        # print(stats.anderson(resid,dist='norm'))
    axes1[-1, -1].axis('off')
    axes2[-1, -1].axis('off')
    lgd1 = fig1.legend(handles = [mean, std], bbox_to_anchor=(0.55, 0.4), loc=2, fontsize=14)
    fig1.tight_layout()
    fig1.subplots_adjust(top=0.85)
    TIFF_exporter(fig1, 'residuals_dist_{}_{}'.format(model_name, DATASET), fig_dir=figPath, extra_artists=(lgd1,))

    lgd2 = fig2.legend(handles = [theoretical_mean, sample_mean], bbox_to_anchor=(0.55, 0.4), loc=2, fontsize=14)
    fig2.tight_layout()
    fig2.subplots_adjust(top=0.85)
    TIFF_exporter(fig2, 'residuals_qq_{}_{}'.format(model_name, DATASET), fig_dir=figPath, extra_artists=(lgd2,))

    posterior = {}
    prior = {}
    entropy = {}
    bins = {}
    fig3, axes3 = plt.subplots(ceil(len(config['parameters'])/3),3, figsize=(7,8))
    i = 0
    for k,v in config['parameters'].items():
        ax = axes3[i//3][i%3]

        prior[k], bins[k] = np.histogram(np.random.uniform(v[1][0], v[1][1], 5000), 50, density=True)
        posterior[k], _ = np.histogram(df_post[k].values, bins=bins[k], density=True)

        entropy[k] = stats.entropy(posterior[k], prior[k])
        line_post = ax.bar(bins[k][:-1], posterior[k], width = bins[k][1]-bins[k][0], align='edge', label='Posterior')
        line_prior = ax.bar(bins[k][:-1], prior[k], width = bins[k][1]-bins[k][0], align='edge', alpha=.75, label='Prior')
        #ax.text(0.7,0.965, "Entropy: {:.3g}".format(entropy[k]), transform=ax.transAxes, size=16)
        ax.set_title("K-L Divergence: {:.3g}".format(entropy[k]), y=1.01, fontsize=12)
        ax.set_xlabel(k)

        fig3.tight_layout()
        i+=1

    n_emptyAxes = 3-len(config['parameters'])%3
    if n_emptyAxes > 0:
        for n in range(1, n_emptyAxes+1):
            axes3[-1, int(-1*n)].axis('off')

    # axes[-1, -2].axis('off')
    # axes3[-1, -1].axis('off')

    lgd3 = fig3.legend(handles=[line_post, line_prior], bbox_to_anchor=(0.7, 0.2), loc=2, fontsize=12)

    TIFF_exporter(fig3, 'kl_div_{}_{}'.format(model_name, DATASET), fig_dir=figPath, extra_artists=(lgd3,))
    plt.close('all')

Working on (bph1, LWP475)
	Running SQL query
	Sampling from the posterior 5000 times.
	Sample 0, idx:1998


KeyboardInterrupt: 

# Residuals #
We can also look at the residuals directly. We can see from both the time series and the distributions of the residuals that they are generally normally distributed aorund 0, with the exception of HbT which is centred roughly on -1.5, showing the general under estimation of the signal. 

In [ ]:
# g = sns.FacetGrid(data=residuals, row = 'Signal', height = 4, aspect = 3, sharey=False,sharex=False)



## Entropy of prior to posterior ##

We need to sample $N_{limit}$ times from the prior distributions for each parameter. $N_{limit}$ is the number of samples in our posterior, in this case that is 3000. 

We then need to bin our posteriors and priors and the calculate the divergence.

In [ ]:
n_emptyAxes = 3-len(config['parameters'])%3
if n_emptyAxes > 0:
    for n in range(1, n_emptyAxes+1):
        print(n)
        #axes[-1, int(-1*n)].axis('off')

In [ ]:
x = (1,2)
print("{},{}".format(*x))